# Data cleaning & validation

In [1]:
import pandas as pd
%matplotlib inline
import re
import string
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

### Load the Dataset

In [2]:
# Experienced an encoding error when loading the file.
# Will have to test to determine the files encoding so that it can be decoded.
# Installed & Ran chardet: the universal character encoding detector in command line
# Found .csv file to be Windows-1254 with confidence 0.49887422177203866
# The detected encoding didn't work, I tried a few other and found the correct one ISO-8859-1.

# Dataset read succesfully
data = pd.read_csv("WELLCOME_APC.csv", encoding='ISO-8859-1')
data.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [3]:
# Get column names / types
print(data.dtypes)
print()
print(data.info())

PMID/PMCID                                             object
Publisher                                              object
Journal title                                          object
Article title                                          object
COST (£) charged to Wellcome (inc VAT when charged)    object
dtype: object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB
None


### 1. Determine the five most common journals and the total articles for each.

First we will need to clean up the journal names.

Due to the fact that the readme file included with the dataset indicated that name authority control was not imposed on "Journal title".

In [4]:
# Fill the Nan to avoid errors
data['Journal title'] = data['Journal title'].fillna('')
# Remove numbers from journal title
data['journal_cleaned'] = data['Journal title'].str.replace('\d+', '')
# Remove punctuation from journal
data['journal_cleaned'] = data['journal_cleaned'].str.replace('[^\w\s]','')
# Remove leading/trailing spaces from journal
data['journal_cleaned'] = data['journal_cleaned'].str.strip()
# Remove Stop Words from journal
data['journal_cleaned'] = data['journal_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
# Change journal to title case
data['journal_cleaned'] = data['journal_cleaned'].str.upper()

#top 5 Journals and number of articles for each
data['journal_cleaned'].value_counts().head(6)

PLOS ONE                        191
JOURNAL BIOLOGICAL CHEMISTRY     51
NEUROIMAGE                       29
NUCLEIC ACIDS RESEARCH           26
PLOS PATHOGENS                   24
PLOS GENETICS                    24
Name: journal_cleaned, dtype: int64

In [5]:
data['journal_cleaned']
top_5 = ['PLOS ONE', 'JOURNAL BIOLOGICAL CHEMISTRY', 'NEUROIMAGE', 'NUCLEIC ACIDS RESEARCH', 'PLOS GENETICS', 'PLOS PATHOGENS']
data.loc[~data['journal_cleaned'].isin(top_5), 'journal_cleaned'] = "Other"
print('The Top 5 Journals and total Articles for each:')
print('(with a tie for 5th)')
data['journal_cleaned'].value_counts()

The Top 5 Journals and total Articles for each:
(with a tie for 5th)


Other                           1782
PLOS ONE                         191
JOURNAL BIOLOGICAL CHEMISTRY      51
NEUROIMAGE                        29
NUCLEIC ACIDS RESEARCH            26
PLOS PATHOGENS                    24
PLOS GENETICS                     24
Name: journal_cleaned, dtype: int64

### 2. For each Journal calculate the mean, median, and standard deviation of the open-access cost per article.

As we know from our value type analysis above. the cost values are objects, so we will need to change them to a numeric type.

In [46]:
# Cost
# Create a new cost column in a numeric type
# Recieved an error and found a single entry of '1674$'. This was preventing us from changing the type to numeric. Solution was adding 'errors='coerce'' to change that to the missing value it is.
data['cost'] = pd.to_numeric(data['COST (£) charged to Wellcome (inc VAT when charged)'].str.strip('£'), errors='coerce')

# display error
data['COST (£) charged to Wellcome (inc VAT when charged)'].loc[178]

'1674$'

In [8]:
# filter on journal to create seperate dataframes
plos_one = data.loc[data['journal_cleaned'] == 'PLOS ONE']
bio_chem = data.loc[data['journal_cleaned'] == 'JOURNAL BIOLOGICAL CHEMISTRY']
neuroimage = data.loc[data['journal_cleaned'] == 'NEUROIMAGE']
nucleic_acid = data.loc[data['journal_cleaned'] == 'NUCLEIC ACIDS RESEARCH']
plos_genetics = data.loc[data['journal_cleaned'] == 'PLOS GENETICS']
plos_pathogens = data.loc[data['journal_cleaned'] == 'PLOS PATHOGENS']

In [45]:
# function to print the answers to our question.
def journal_calc(journal):
    print(''.join(journal['journal_cleaned'].unique()))
    print('Mean: ' + str(journal['cost'].mean()))
    print('Median: ' + str(journal['cost'].median()))
    print('Standard Deviation: ' + str(journal['cost'].std()))

In [49]:
#run our journals through our journal_calc funciton above.
separate_journals = [plos_one, bio_chem, neuroimage, nucleic_acid, plos_genetics, plos_pathogens]

for journal in separate_journals:
    journal_calc(journal)
    print()

PLOS ONE
Mean: 38552.857905759156
Median: 897.61
Standard Deviation: 188531.40909246643

JOURNAL BIOLOGICAL CHEMISTRY
Mean: 20990.932941176467
Median: 1286.86
Standard Deviation: 139830.9526257881

NEUROIMAGE
Mean: 2215.168275862069
Median: 2326.43
Standard Deviation: 266.65394691928987

NUCLEIC ACIDS RESEARCH
Mean: 1149.0
Median: 852.0
Standard Deviation: 442.9404474644419

PLOS GENETICS
Mean: 84839.43500000001
Median: 1718.3899999999999
Standard Deviation: 281865.7077944168

PLOS PATHOGENS
Mean: 112525.16277777778
Median: 1684.0
Standard Deviation: 322866.0785688621



Several of the Journals have to an extremely high mean. We will need to dig deeper into the cost values to see what is going on.

In [50]:
# analyze Cost
data.describe()

#data = data.groupby('journal_cleaned').agg({'cost': 'sum'})
#data.sort_values(by='cost', ascending=False).head()

,cost
count,2114.000000
mean,24206.441892
std,147300.992149
min,0.000000
25%,1280.000000
50%,1889.950000
75%,2322.315000
max,999999.000000


The top 25 percentile seems to contain some null values disguised as 999999

In [51]:
data[data['cost'] > 999998].head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged),journal_cleaned,cost
149,PMC3234811,ASBMB,Journal of Biological Chemistry,Picomolar nitric oxide signals from central ne...,£999999.00,JOURNAL BIOLOGICAL CHEMISTRY,999999.0
227,3708772,BioMed Central,BMC Genomics.,"Phenotypic, genomic, and transcriptional chara...",£999999.00,Other,999999.0
277,PMC3668259,BMC,Trials,Community resource centres to improve the heal...,£999999.00,Other,999999.0
358,PMC3219211,Cambridge University Press,Expert Reviews in Molecular Medicine,Pharmacological targets in the ubiquitin syste...,£999999.00,Other,999999.0
404,PMC3533396,Company of Biologists,Journal of Cell Science,PKA isoforms coordinate mRNA fate during nutri...,£999999.00,Other,999999.0


It doesn't look like the nulls are isolated to a single journal

In [34]:
data[data['cost'] < 999999].sort_values(by='cost', ascending=False).head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged),journal_cleaned,cost
1987,PMC3664409\n\n,Wiley,Movement Disorders,Limb amputations in fixed dystonia: a form of ...,£201024.00,Other,201024.0
1470,3547931,Public Library of Science,PLoS One,Reducing stock-outs of life saving Malaria Com...,£192645.00,PLOS ONE,192645.0
986,NaN,MacMillan,,Fungal Disease in Britain and the United State...,£13200.00,Other,13200.0
1619,543219,public.service.co.uk,Public Service Review,Laboratory Science in Tropical Medicine,£6000.00,Other,6000.0
800,PMID: 23041239 /PMCID: PMC3490334,Elsevier,The Lancet Neurology,Genetic risk factors for ischaemic stroke and ...,£5760.00,Other,5760.0


There are only 3 outliers on the high end outside of the 999999 nulls. After that point the cost lowers back down to a sub £6000 range.

There appears to be a significant number of £999,999 values for the cost collumn. A possible reason for that could be the result of there being no value for the specific article when the data was entered.

Lets filter for cost values < 999999 and run our calculations again to see if they are more accurate.

In [60]:
clean_journals = []

def filter_cost(journal):
    journal = journal[journal['cost'] < 999999]
    return clean_journals.append(journal)

for journal in separate_journals:
    filter_cost(journal)

In [62]:
for journal in clean_journals:
    journal_calc(journal)
    print()

PLOS ONE
Mean: 1976.1025
Median: 896.975
Standard Deviation: 14134.534902134803

JOURNAL BIOLOGICAL CHEMISTRY
Mean: 1410.7716
Median: 1281.4699999999998
Standard Deviation: 424.60878010263156

NEUROIMAGE
Mean: 2215.168275862069
Median: 2326.43
Standard Deviation: 266.65394691928987

NUCLEIC ACIDS RESEARCH
Mean: 1149.0
Median: 852.0
Standard Deviation: 442.9404474644419

PLOS GENETICS
Mean: 1643.1109090909092
Median: 1712.73
Standard Deviation: 153.36682533527224

PLOS PATHOGENS
Mean: 1590.933125
Median: 1623.505
Standard Deviation: 183.17405823310062



Above are more accurate estimates for the top Journals.

### Futher Cleaning & Analysis
Additional analysis could be done by analyzing the articles by subject as well as by publisher, or even to see if different subjects have higher cost than others.